In [1]:
import numpy as np
import pandas as pd

from sklearn.decomposition import TruncatedSVD

In [63]:
df_recsys = pd.read_csv('./../data/sb_restaurants_clean.csv', index_col=0)
df_recsys.head(15)

,user_id,business_id,stars,name
0,IlWLPCRQp8iqX0X-ExQccQ,bdfZdB2MTXlT6-RBjSIpQg,4.0,Pho Bistro
1,WSMIRegvrsEgFGEraf_LwQ,bdfZdB2MTXlT6-RBjSIpQg,5.0,Pho Bistro
2,2odfcvFhkb8SedI3vCLpmQ,bdfZdB2MTXlT6-RBjSIpQg,5.0,Pho Bistro
3,OEKu0Rts0spELpbnucDKmA,bdfZdB2MTXlT6-RBjSIpQg,5.0,Pho Bistro
4,4MSEWnnxKhNdh7GgNHM-YQ,bdfZdB2MTXlT6-RBjSIpQg,4.0,Pho Bistro
5,gJyqSThsQDT-04VE2gR1TQ,bdfZdB2MTXlT6-RBjSIpQg,4.0,Pho Bistro
6,sR3Na7Z0-9-wuv9cU8TW1w,bdfZdB2MTXlT6-RBjSIpQg,1.0,Pho Bistro
7,GVoSyA04GQTLZb77Bz_kvw,bdfZdB2MTXlT6-RBjSIpQg,1.0,Pho Bistro
8,qb0tcchdlgGBndTMdjMvoA,bdfZdB2MTXlT6-RBjSIpQg,4.0,Pho Bistro
9,8eOQq1bHixOS75rfucPwGw,bdfZdB2MTXlT6-RBjSIpQg,1.0,Pho Bistro


In [3]:
df_recsys.shape 

(37830, 4)

Iniciamos por elegir a algun usuario aleatorio y vemos qué restaurantes ha calificado

In [78]:
# Choose a random user
idx = (df_recsys['user_id']== '4MSEWnnxKhNdh7GgNHM-YQ')

# print user's reviews
print('Total de restaurantes calificados por el usuario: ',idx.sum())
df_recsys[idx]

Total de restaurantes calificados por el usuario:  5


,user_id,business_id,stars,name
4,4MSEWnnxKhNdh7GgNHM-YQ,bdfZdB2MTXlT6-RBjSIpQg,4.0,Pho Bistro
1586,4MSEWnnxKhNdh7GgNHM-YQ,ju4YP8SLdR_BmWr_-Xh83Q,3.0,Phamous Cafe
10500,4MSEWnnxKhNdh7GgNHM-YQ,zJpZ-uQ_F0XVgK1u98uWSg,5.0,Gloria's Gourmet Kitchen
25128,4MSEWnnxKhNdh7GgNHM-YQ,IoAubn7CpU2rqcURxUo1MA,4.0,Barb's Pies
34522,4MSEWnnxKhNdh7GgNHM-YQ,yM7eA2uuH3Ch7OYVj3PKSw,5.0,Wabi Sabi


Ahora generamos una matriz de utilidad, la cual nos proporciona la calificación que cada usuario le da a cada restaurante que ha visitado.

In [5]:
# Create a utility matrix
UtlMtrx = df_recsys.pivot_table(values='stars', index='user_id', columns='name', fill_value=0)

UtlMtrx.head(10) 

name,101 Deli,1114 Sports Bar & Games,4 Eggs & Pizza,A Slice of Woodstock's,AH Juice Organics,AR Catering,ASIE,Aksum Restaurant,Aladdin Cafe,Alcazar Tapas Bar,...,Yona Redz,Your Choice,Your Place Thai Restaurant,Yume Sushi Japanese Restaurant,Z's - Taphouse & Grill,Zaytoon,Zen Yai Thai Cuisine,Zizzo's Coffeehouse,Zodo's Bowlero Bowling & Beyond,Zookers Cafe
user_id,,,,,,,,,,,,,,,,,,,,,
---zemaUC8WeJeWKqS6p9Q,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,0,0,0
--2F5G5LKt3h2cAXJbZptg,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,0,0,0
--7XOV5T9yZR5w1DIy_Dog,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,0,0,0
--8YG4BoOWFGjdh9Fxop-w,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,0,0,0
--_QWYpJ2In2IgW_bouSuQ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,0,0,0
--kinfHwmtdjz03g8B8z8Q,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,0,0,0
-0-Cbo-4YzKmoJXpnckwbg,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,0,0,0
-0-TtVhV4PIUoDpUCOC0uQ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,0,0,0
-06H6BQ5cS5A5Jpd_iiwCA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,0,0,0


- Observamos que una gran cantidad de valores en la matriz son 0. De los 659 restaurantes es de esperarse que los usuarios solo han evaluado unos cuantos, por eso la gran cantidad de 0´s. 
- En álgebra lineal este tipo de matriz se le conoce como matriz dispersa (sparse matrix).

In [13]:
print('Total de elementos de la matriz de utilidad: %d' % (UtlMtrx.size))
print('Total de elementos diferentes de cero: %d' % (np.count_nonzero(UtlMtrx)))
print('Porcentaje de elementos diferentes de cero: %.1f%%' % (100 * np.count_nonzero(UtlMtrx) / UtlMtrx.size))

Total de elementos de la matriz de utilidad: 15465412
Total de elementos diferentes de cero: 37166
Porcentaje de elementos diferentes de cero: 0.2%


Construiremos el sistema de recomendación utilizando el metódo de reducción de dimensionalidad SVD (Singular Value Decomposition) o Descomposicion de Valores Singulares. 

Este tipo de solución es utilizado comunmente como un primer sistema de recomendación, basado en las calificaciones que los usuarios han asignado en el pasado.

Para obtener la información relacionada con los restaurantes, tomamos la transpuesta de la matriz de utilidad.

In [6]:
# Transpose the utility matrix
X = UtlMtrx.T
X.shape

(659, 23468)

In [41]:
SVD = TruncatedSVD(n_components=658, random_state=42)  
SVD.fit(X)

num_sv = 7

print('Cantidad de información simplificada con los primeros %d vectores singulares:' % num_sv)
print('%.1f%%' %  (100 * (1- (SVD.singular_values_[0:num_sv]).sum() / (SVD.singular_values_).sum())))

Cantidad de información simplificada con los primeros 7 vectores singulares:
95.8%


### 

In [42]:
num_sv = 7

SVD = TruncatedSVD(n_components=num_sv, random_state=1)

resultant_matrix = SVD.fit_transform(X)
resultant_matrix.shape

(659, 7)

In [98]:
# Pearson correlation matrix
corrMtx = np.corrcoef(resultant_matrix)
corrMtx.shape  
corrMtx[0:5,0:5]


array([[1.        , 0.55515233, 0.41973699, 0.65740579, 0.64679698],
       [0.55515233, 1.        , 0.98189643, 0.18109368, 0.94777665],
       [0.41973699, 0.98189643, 1.        , 0.05835276, 0.92436228],
       [0.65740579, 0.18109368, 0.05835276, 1.        , 0.11235041],
       [0.64679698, 0.94777665, 0.92436228, 0.11235041, 1.        ]])

Elegimos un restaurante para ver su correlación con todos los demás en la matriz

In [130]:
# Look for The Lark index

liked = 'The Lark'

names = UtlMtrx.columns 
names_list = list(names)
id_liked = names_list.index(liked)

id_liked

582

In [131]:
# Check the correlation with some restaurants
corr_recom = corrMtx[id_liked]
print(corr_recom.shape)
corr_recom[0:10]

(659,)


array([0.66494485, 0.93200037, 0.88695815, 0.37695801, 0.91948546,
       0.12857906, 0.95105676, 0.89818658, 0.9203617 , 0.90324919])

Comprobamos que la correlacion de el restaurante elegido consigo mismo es = 1

In [132]:
# Check restaurant correlation with itself
corr_recom[id_liked]

0.9999999999999999

### Obtenemos las recomendaciones de los restaurantes seleccionados

#### *Ejemplo 1*:

In [137]:
# Restaurant 'The Lark'

print('Recomendaciones: ')
list(names[(corr_recom > .97) & (corr_recom < .99)]) 

Recomendaciones: 


["Ana's Taco Bar",
 'Beans BBQ and Catering',
 'Biergarten',
 'Bouchon',
 'Cajé Coffee Roasters - Haley St',
 'Coast & Olive',
 'Dawn Patrol',
 'Intermezzo By Wine Cask',
 "Jack's Bistro & Famous Bagels",
 'Lucky Penny',
 'Malibu Farm at Miramar',
 'Merci To Go',
 'Montecito Gourmet by Village Cheese & Wine',
 'Sama Sama Kitchen',
 'Shintori Sushi Factory',
 'The Harbor Restaurant']

Ordenamos de manera descendente de acuerdo a la correlación 

In [139]:
ids = (corr_recom > .97) & (corr_recom < 0.99)
tmp = list()

for i in range(len(names[ids])):
    tmp.append((corr_recom[ids][i], names[ids][i]))

sorted(tmp, key=lambda x:x[0], reverse=True) 

[(0.9897254070135815, 'Shintori Sushi Factory'),
 (0.9864394995323224, 'Bouchon'),
 (0.9826120609276208, 'Malibu Farm at Miramar'),
 (0.9818476588916369, 'The Harbor Restaurant'),
 (0.9812980532356922, 'Biergarten'),
 (0.981057146332992, 'Lucky Penny'),
 (0.9802099655055196, 'Intermezzo By Wine Cask'),
 (0.9796908115707111, 'Merci To Go'),
 (0.977241751437632, "Ana's Taco Bar"),
 (0.9758169024694828, 'Beans BBQ and Catering'),
 (0.9734240399514472, 'Cajé Coffee Roasters - Haley St'),
 (0.9725699971747017, 'Coast & Olive'),
 (0.9715408853272126, 'Dawn Patrol'),
 (0.9714517566101355, 'Sama Sama Kitchen'),
 (0.9711297352604064, 'Montecito Gourmet by Village Cheese & Wine'),
 (0.9703032649598626, "Jack's Bistro & Famous Bagels")]

_ _ _

#### *Ejemplo 2:*

In [140]:
# Restaurant 'Sandbar'
liked = 'Sandbar'

id_liked = names_list.index(liked)
corr_recom = corrMtx[id_liked]
print('Recomendaciones: ')
list(names[(corr_recom > .97) & (corr_recom < 0.99)])

Recomendaciones: 


['Bluewater Grill - Santa Barbara',
 'Breakwater Restaurant',
 "Caruso's",
 'New Si Chuan Garden',
 'Opal Restaurant & Bar',
 'Roof Top Bistro & Bar',
 'Santa Barbara FisHouse',
 'The Funk Zone',
 'Your Choice']

In [141]:
ids = (corr_recom > .97) & (corr_recom < 0.99)
tmp = list()

for i in range(len(names[ids])):
    tmp.append((corr_recom[ids][i], names[ids][i]))

sorted(tmp, key=lambda x:x[0], reverse=True) 

[(0.9870209870983876, 'Opal Restaurant & Bar'),
 (0.986307556971767, "Caruso's"),
 (0.9844777407402455, 'The Funk Zone'),
 (0.9786889568146974, 'Breakwater Restaurant'),
 (0.9784217518060125, 'Santa Barbara FisHouse'),
 (0.9778500625915235, 'Roof Top Bistro & Bar'),
 (0.9770293816764639, 'Your Choice'),
 (0.9748661767806976, 'New Si Chuan Garden'),
 (0.9700119597403826, 'Bluewater Grill - Santa Barbara')]

_ _ _

#### *Ejemplo 3*


In [100]:
# Restaurant 'La Super-Rica Taqueria'
liked = 'La Super-Rica Taqueria'

id_liked = names_list.index(liked)
corr_recom = corrMtx[id_liked]
print('Recomendaciones: ')
list(names[(corr_recom > .97) & (corr_recom < 1)])

Recomendaciones: 


['Alessia Patisserie & Cafe ',
 'Arigato Sushi',
 'Barbareño',
 'Beans BBQ and Catering',
 'Biergarten',
 "Bree'osh",
 'Cajé Coffee Roasters - Haley St',
 'Ca’Dario - Montecito',
 'Coast & Olive',
 'Dawn Patrol',
 'Empty Bowl Gourmet Noodle Bar',
 'Helena Avenue Bakery',
 'Hook And Press Donuts',
 'Hotdog Mini Mart',
 'M. Special Brewing Company',
 "Matty's Hot Chicken",
 'Merci To Go',
 'Modern Times Academy of Recreational Sciences',
 'Olio E Limone Ristorante',
 'Palihouse Santa Barbara',
 'Sama Sama Kitchen',
 'Santa Barbara Public Market',
 'Scarlett Begonia',
 'Shaker Mill',
 'Stonehouse Restaurant',
 'Tre Lune',
 'Tyger Tyger']

In [101]:
ids = (corr_recom > .97) & (corr_recom < 1)
tmp = list()

for i in range(len(names[ids])):
    tmp.append((corr_recom[ids][i], names[ids][i]))

sorted(tmp, key=lambda x:x[0], reverse=True) 

[(0.9976673700224502, 'Empty Bowl Gourmet Noodle Bar'),
 (0.9923020640278319, 'Helena Avenue Bakery'),
 (0.9886038360634839, 'Sama Sama Kitchen'),
 (0.9874513345615387, 'Scarlett Begonia'),
 (0.9855324088293036, 'Ca’Dario - Montecito'),
 (0.9855036854186333, 'Olio E Limone Ristorante'),
 (0.9854917131005185, 'Beans BBQ and Catering'),
 (0.9808548046705531, 'Santa Barbara Public Market'),
 (0.9804824693096913, 'Modern Times Academy of Recreational Sciences'),
 (0.9803076168902964, 'Cajé Coffee Roasters - Haley St'),
 (0.9791048015557641, 'Tyger Tyger'),
 (0.9782302416419174, 'Barbareño'),
 (0.9779941499149726, 'Alessia Patisserie & Cafe '),
 (0.9775090339539287, 'Tre Lune'),
 (0.9772521039973178, 'Shaker Mill'),
 (0.9770921739300092, 'Coast & Olive'),
 (0.9756743097794676, 'M. Special Brewing Company'),
 (0.9750232570217748, 'Merci To Go'),
 (0.9749660609090739, 'Arigato Sushi'),
 (0.9736917201722163, "Matty's Hot Chicken"),
 (0.9732983763966024, 'Hotdog Mini Mart'),
 (0.972977010553168